# Single-site

author: laquitainesteeve@gmail.com  

execution time: 1 min

Special hardware: on CPU, does not require GPU.

## Setup

Create and activate `demo` virtual environment kernel `demo` from (envs/demo.yml)

In [6]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
from dandi.dandiapi import DandiAPIClient
import spikeinterface.extractors as se
import spikeinterface
from matplotlib import pyplot as plt;
import spikeinterface.preprocessing as spre
print("spikeinterface", spikeinterface.__version__)

# dandiset parameters
dandiset_id = '001250'
filepath = 'sub-vivo-horvath-depth-2/sub-vivo-horvath-depth-2_ecephys.nwb'

# project path
proj_path = "/home/steeve/steeve/epfl/code/spikebias"
os.chdir(proj_path)

# import spikebias
from src.nodes.validation import noise

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
spikeinterface 0.101.2


## Load demo

In [ ]:
%%time

# get the extractors from Dandi archive
with DandiAPIClient() as client:
    asset = client.get_dandiset(dandiset_id, 'draft').get_asset_by_path(filepath)
    s3_path = asset.get_content_url(follow_redirects=1, strip_query=True)
Recording = se.NwbRecordingExtractor(file_path=s3_path, stream_mode="remfile")

# get only 10 minutes of data
Recording = Recording.frame_slice(start_frame=0, end_frame=Recording.sampling_frequency*600)

# preprocess
Recording = spre.highpass_filter(Recording, freq_min=300)
Recording = spre.common_reference(Recording, reference="global", operator="median")

# load voltage traces 
traces = Recording.get_traces()
traces_in_uV = Recording.get_traces(return_scaled=True)

# report metadata
print("s3_path:", s3_path)
print('\n Recording extractor:', Recording)
print("trace data shape:", traces.shape)
print("gain_to_uV:", Recording.get_property('gain_to_uV'))
print("offset_to_uV: ", Recording.get_property('offset_to_uV'))

* from stimulus start (STIM_START= 20110000 at the 5000th stimulus epoch, t=0 ms) to 1000 ms after.
* plot trace from site 143 in layer 5, same as for the spontaneous model.

## Plot single-trace and calculate noise (MAD)

In [ ]:
%%time 

# parameters
LAYER = 'L2_3'
STIM_START = 0
SITE_IX = 21
SFREQ = int(Recording.get_sampling_frequency())

# get one second trace to plot
onesecond = np.arange(STIM_START, STIM_START + SFREQ, 1).astype(int)
layers = Recording.get_property("layers")
L5_site = np.where(layers == LAYER)[0][SITE_IX]
fig_data = traces[onesecond, L5_site]

# plot
YMAX = 60
fig, ax = plt.subplots(1, 2, figsize=(4,2))

# raw ---------------------------------
ax[0].plot(fig_data, color="k")

# legend
ax[0].spines[["right", "top"]].set_visible(False)
ax[0].set_ylabel("Voltage (\u03bcV)")
ax[0].set_xlabel("Time after stimulus (ms)")
ax[0].set_yticks([-120, 0, YMAX], [-120, 0, YMAX])
ax[0].set_ylim([-120, YMAX])
ax[0].set_xticks([0, SFREQ / 2, SFREQ], [0, 500, 1000])
ax[0].set_xlim([0, SFREQ])

# disconnected
ax[0].spines["bottom"].set_position(("axes", -0.05))
ax[0].yaxis.set_ticks_position("left")
ax[0].spines["left"].set_position(("axes", -0.05))

# report noise (min(MAD) over segments of 1 sec) here
wind_end = int(np.floor(len(traces)/SFREQ))
bkg_noise = noise.measure_trace_noise(traces[onesecond, L5_site], SFREQ, wind_end)
print("nb of 1-sec windows:", len(bkg_noise))
print("Background noise level in uV (min MAD over windows):", min(bkg_noise))



# scaled to uV ---------------------------------
fig_data = traces_in_uV[onesecond, L5_site]

# plot figure
ax[1].plot(fig_data, color="k")

# report noise (min(MAD) over segments of 1 sec) here
wind_end = int(np.floor(len(traces_in_uV)/SFREQ))
bkg_noise = noise.measure_trace_noise(traces_in_uV[onesecond, L5_site], SFREQ, wind_end)
print("nb of 1-sec windows:", len(bkg_noise))
print("Background noise level in uV (min MAD over windows):", min(bkg_noise))

IndexError: index 21 is out of bounds for axis 0 with size 8